In [50]:
import os
import io
import struct
from typing import List, Tuple, Optional
from enum import Enum

from blowfish import Blowfish

In [51]:
class PackEntryType(Enum):
  Nop = 0
  Folder = 1
  File = 2

In [52]:
def generate_final_blowfish_key(password: str, salt: bytes) -> bytes:
    """
    Reproduces the 'GenerateFinalBlowfishKey' logic in C#.
    """
    # 1) Limit key length to max of 56
    plain_key_length = min(len(password), 56)

    # 2) Convert password to ASCII bytes
    a_key = password.encode("ascii")

    # 3) Create a 56-byte base key buffer
    b_key = bytearray(56)
    
    # Copy salt into b_key
    # Equivalent to `Array.ConstrainedCopy(salt, 0, bKey, 0, salt.Length)`
    b_key[:len(salt)] = salt

    # 4) Generate the final blowfish key by XOR-ing
    #    the ASCII password bytes with the corresponding part of b_key.
    bf_key = bytearray(plain_key_length)
    for x in range(plain_key_length):
        bf_key[x] = a_key[x] ^ b_key[x]

    return bytes(bf_key)

In [53]:
def read_string_with_length(stream, byte_count: int) -> str:
    """
    Read 'byte_count' bytes, then decode using self._encoding,
    stopping at the first 0 (null terminator) if present.
    """
    buffer = stream.read(byte_count)

    # Find null terminator offset (if any)
    terminator_offset = byte_count
    for i in range(byte_count):
        if buffer[i] == 0:
            terminator_offset = i
            break

    return buffer[:terminator_offset].decode('ascii', errors='replace')

In [54]:
data_pk2_path = os.path.join(os.getcwd(), 'data', 'Data.pk2')
media_pk2_path = os.path.join(os.getcwd(), 'data', 'Media.pk2')
print(data_pk2_path)
print(media_pk2_path)

c:\Users\htdun\Desktop\workspace\pk2-extractor\data\Data.pk2
c:\Users\htdun\Desktop\workspace\pk2-extractor\data\Media.pk2


In [55]:
pk2key = '169841'
salt = [0x03, 0xF8, 0xE4, 0x44, 0x88, 0x99, 0x3F, 0x64, 0xFE, 0x35]

In [56]:
key = generate_final_blowfish_key(pk2key, bytes(salt))
key

b'2\xce\xdd|\xbc\xa8'

In [57]:
blowfish = Blowfish()
blowfish.Initialize(key)

In [58]:
_media_file_stream = open(media_pk2_path, 'rb')

In [59]:
signature = _media_file_stream.read(30)
version =  struct.unpack('<i', _media_file_stream.read(4))[0]
encrypted = _media_file_stream.read(1)
encryption_checksum = _media_file_stream.read(16)
payload = _media_file_stream.read(205)


In [60]:
Header = {
    'signature': signature,
    'version': version,
    'encrypted': encrypted,
    'encryption_checksum': encryption_checksum,
    'payload': payload
}

In [61]:
blowfish_checksum_decoded = "Joymax Pak File"

In [62]:
if blowfish and encrypted == b'\x01':
    temp_checksum = blowfish.Encode(blowfish_checksum_decoded.encode('ascii'))
    if temp_checksum is None or temp_checksum[0] != encryption_checksum[0] or temp_checksum[1] != encryption_checksum[1] or temp_checksum[2] != encryption_checksum[2]:
        raise Exception('Failed to open JoymaxPackFile: The password or salt is wrong.')

In [63]:
def read_block_at(position: int):
    _media_file_stream.seek(position, io.SEEK_SET)
    buffer = _media_file_stream.read(128 * 20)
    if blowfish is not None:
      entry_buffer = io.BytesIO(blowfish.Decode(buffer))
    else:
      entry_buffer = io.BytesIO(buffer)

    entries = []
    for _ in range(20):
      entry = {
        "Type": PackEntryType(entry_buffer.read(1)[0]),
        "Name": read_string_with_length(entry_buffer, 89).rstrip('\0'),
        "CreationTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "ModifyTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataPosition": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataSize": struct.unpack('<i', entry_buffer.read(4))[0],
        "NextBlock": struct.unpack('<q', entry_buffer.read(8))[0],
        "Payload": entry_buffer.read(2)
      }
      entries.append(entry)
      # print(entry)

    return {
      "Position": position,
      "Entries": entries
    }


def read_blocks_at(position: int):
    result = []

    block = read_block_at(position)
    result.append(block)

    if block["Entries"][19]["NextBlock"] > 0:
        result.extend(read_blocks_at(block["Entries"][19]["NextBlock"]))

    return result

In [64]:
Root = read_blocks_at(256)

In [65]:
blocks_in_memory = {
  "": Root
}

In [66]:
blocks_in_memory

{'': [{'Position': 256,
   'Entries': [{'Type': <PackEntryType.Folder: 1>,
     'Name': '.',
     'CreationTime': 133336205048819447,
     'ModifyTime': 133336205048819447,
     'DataPosition': 256,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'acobject',
     'CreationTime': 133336205048829450,
     'ModifyTime': 133336205048829450,
     'DataPosition': 2816,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'config',
     'CreationTime': 133336205049179438,
     'ModifyTime': 133336205049179438,
     'DataPosition': 35136,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'Effect',
     'CreationTime': 133336205049679437,
     'ModifyTime': 133336205049679437,
     'DataPosition': 41896,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
   

In [67]:
server_dep = os.path.join('server_dep', 'silkroad', 'textdata')
server_dep

'server_dep\\silkroad\\textdata'

In [68]:
character_text_data = dict()

## TextUISystem.txt

In [69]:
character_data = os.path.join(server_dep, 'CharacterData.txt')
character_data

'server_dep\\silkroad\\textdata\\CharacterData.txt'

In [70]:
def get_entry_buffer(file_path: str) -> Optional[dict]:
  parent_folder_path = os.path.dirname(file_path)
  file_name = os.path.basename(file_path)
  if parent_folder_path not in blocks_in_memory:
    paths = parent_folder_path.split(os.path.sep)
    blocks = blocks_in_memory[""]
    current_path = ""

    for sub_folder_name in paths:
        for block in blocks:
            entries = block["Entries"]
            entry = next((e for e in entries if e["Name"] == sub_folder_name and e["Type"] == PackEntryType.Folder), None)
            if entry is None:
                continue
            
            current_path = os.path.join(current_path, entry["Name"])
            if current_path in blocks_in_memory:
                blocks = blocks_in_memory[current_path]
                break
            
            blocks = read_blocks_at(entry["DataPosition"])
            blocks_in_memory[current_path] = blocks
            break
        
  root = blocks_in_memory[parent_folder_path]
  entry = None
  for entries in root:
    entry = next((x for x in entries["Entries"] if x["Name"] == file_name.lower()), None)
    if entry:
      break

  _media_file_stream.seek(entry["DataPosition"], io.SEEK_SET)
  # _media_file_stream.read(entry["DataSize"])
  buffer = io.BytesIO(_media_file_stream.read(entry["DataSize"])) 

  return buffer
   

def get_lines(file_path: str) -> List[Tuple[str, int]]:

  buffer = get_entry_buffer(file_path)

  text = buffer.read().decode('utf-16', errors="replace")
  lines = text.split('\r\n')
  return lines  


In [71]:
character_data_files = get_lines(character_data)
character_data_files

['CharacterData_10000.txt',
 'CharacterData_10100.txt',
 'CharacterData_10200.txt',
 'CharacterData_10300.txt',
 'CharacterData_14800.txt',
 'CharacterData_14900.txt',
 'CharacterData_15000.txt',
 'CharacterData_15100.txt',
 'CharacterData_15200.txt',
 'CharacterData_15300.txt',
 'CharacterData_15400.txt',
 'CharacterData_15500.txt',
 'CharacterData_15600.txt',
 'CharacterData_15700.txt',
 'CharacterData_15800.txt',
 'CharacterData_15900.txt',
 'CharacterData_16000.txt',
 'CharacterData_16100.txt',
 'CharacterData_16200.txt',
 'CharacterData_16300.txt',
 'CharacterData_16400.txt',
 'CharacterData_16500.txt',
 'CharacterData_16600.txt',
 'CharacterData_16700.txt',
 'CharacterData_16800.txt',
 'CharacterData_16900.txt',
 'CharacterData_17000.txt',
 'CharacterData_17100.txt',
 'CharacterData_17200.txt',
 'CharacterData_17300.txt',
 'CharacterData_17400.txt',
 'CharacterData_17500.txt',
 'CharacterData_17600.txt',
 'CharacterData_17700.txt',
 'CharacterData_17800.txt',
 'CharacterData_1790

In [72]:
for xxx in character_data_files:
  file_name = os.path.join(server_dep, xxx)
  lines = get_lines(file_name)
  for idx, line in enumerate(lines):
    values = line.split('\t')
    if len(values) < 11:
      continue
    Service = values[0]
    ID = values[1]
    CodeName = values[2]
    ObjName = values[3]
    NameStrId = values[5]
    CashItem = values[7]
    Bionic = values[8]
    TypeId1 = values[9]
    TypeId2 = values[10]
    TypeId3 = values[11]
    TypeId4 = values[12]
    Country = values[14]
    Rarity = values[15]
    CanDrop = values[20]
    CanUSe = values[24]
    RequestLevelType1 = values[32]
    RequestLevel1 = values[33]
    RequestLevelType2 = values[34]
    RequestLevel2 = values[35]
    RequestLevelType3 = values[36]
    RequestLevel3 = values[37]
    RequestLevelType4 = values[38]
    RequestLevel4 = values[39]
    Speed1 = values[46]
    Speed2 = values[47]
    AssicFileIcon = values[54]

    Level = values[57]
    CharGender = values[58]
    MaxHealth = values[59]
    MaxMP = values[60]
    InventorySize = values[61]
    CanStore_TID1 = values[62]
    CanStore_TID2 = values[63]
    CanStore_TID3 = values[64]
    CanStore_TID4 = values[65]
    CanBeVehicle = values[66]
    CanControl = values[67]
    DamagePortion = values[68]
    MaxPassenger = values[69]

    IsDimensionPillar = NameStrId == "SN_MOB_GOD_PILLAR"
    IsSummonFlower = CodeName.startswith("STRUCTURE_SUMMON_FLOWER_")
    IsEventMob = CodeName.startswith("MOB_EV")
    IsPandora = TypeId2 == 2 and TypeId3 == 1 and TypeId4 == 5

    character_text_data[ID] = {
      "Service": Service,
      "ID": ID,
      "CodeName": CodeName,
      "ObjName": ObjName,
      "NameStrId": NameStrId,
      "CashItem": CashItem,
      "Bionic": Bionic,
      "TypeId1": TypeId1,
      "TypeId2": TypeId2,
      "TypeId3": TypeId3,
      "TypeId4": TypeId4,
      "Country": Country,
      "Rarity": Rarity,
      "CanDrop": CanDrop,
      "CanUSe": CanUSe,
      "RequestLevelType1": RequestLevelType1,
      "RequestLevel1": RequestLevel1,
      "RequestLevelType2": RequestLevelType2,
      "RequestLevel2": RequestLevel2,
      "RequestLevelType3": RequestLevelType3,
      "RequestLevel3": RequestLevel3,
      "RequestLevelType4": RequestLevelType4,
      "RequestLevel4": RequestLevel4,
      "Speed1": Speed1,
      "Speed2": Speed2,
      "AssicFileIcon": AssicFileIcon,
      "Level": Level,
      "CharGender": CharGender,
      "MaxHealth": MaxHealth,
      "MaxMP": MaxMP,
      "InventorySize": InventorySize,
      "CanStore_TID1": CanStore_TID1,
      "CanStore_TID2": CanStore_TID2,
      "CanStore_TID3": CanStore_TID3,
      "CanStore_TID4": CanStore_TID4,
      "CanBeVehicle": CanBeVehicle,
      "CanControl": CanControl,
      "DamagePortion": DamagePortion,
      "MaxPassenger": MaxPassenger,
      "IsDimensionPillar": IsDimensionPillar,
      "IsSummonFlower": IsSummonFlower,
      "IsEventMob": IsEventMob,
      "IsPandora": IsPandora
    }


In [73]:
character_text_data

{'9900': {'Service': '1',
  'ID': '9900',
  'CodeName': 'COS_GUILD_CH_SOLDIER5_059',
  'ObjName': '???? lv 59',
  'NameStrId': 'SN_COS_GUILD_CH_SOLDIER5',
  'CashItem': '0',
  'Bionic': '1',
  'TypeId1': '1',
  'TypeId2': '2',
  'TypeId3': '3',
  'TypeId4': '5',
  'Country': '0',
  'Rarity': '0',
  'CanDrop': '0',
  'CanUSe': '0',
  'RequestLevelType1': '-1',
  'RequestLevel1': '0',
  'RequestLevelType2': '-1',
  'RequestLevel2': '0',
  'RequestLevelType3': '-1',
  'RequestLevel3': '0',
  'RequestLevelType4': '-1',
  'RequestLevel4': '0',
  'Speed1': '20',
  'Speed2': '50',
  'AssicFileIcon': 'cos\\guild_ch_soldier5_scroll.ddj',
  'Level': '59',
  'CharGender': '0',
  'MaxHealth': '13245',
  'MaxMP': '0',
  'InventorySize': '0',
  'CanStore_TID1': '0',
  'CanStore_TID2': '0',
  'CanStore_TID3': '0',
  'CanStore_TID4': '0',
  'CanBeVehicle': '0',
  'CanControl': '0',
  'DamagePortion': '0',
  'MaxPassenger': '0',
  'IsDimensionPillar': False,
  'IsSummonFlower': False,
  'IsEventMob': F